In [ ]:
!unzip beard.zip

In [ ]:
!unzip gender.v1i.folder.zip

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split



In [ ]:
def load_images_from_folder(folder, label, img_size=(64, 64)):   #Function to Load images and assign labels
    images = []
    labels = []
    if not os.path.exists(folder):
        raise FileNotFoundError(f"Folder not found: {folder}")
    for filename in os.listdir(folder):
        if filename.lower().endswith(".jpg"):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, img_size)
            img = img / 255.0
            images.append(img)
            labels.append(label)
    if not images:
        raise ValueError(f"No valid .jpg images found in folder: {folder}")
    return images, labels



In [ ]:
male_folder = "gender.v1i.folder\train\Male"   # Paths for the datasets
female_folder = "gender.v1i.folder\train\Female"


# Load gender images (0 for male, 1 for female)
male_images, male_labels = load_images_from_folder(male_folder, 0)
female_images, female_labels = load_images_from_folder(female_folder, 1)

# Combining gender data
X_gender = np.array(male_images + female_images)
y_gender = np.array(male_labels + female_labels)

# Splitting the gender data for ML Model
X_train_gender, X_test_gender, y_train_gender, y_test_gender = train_test_split(
    X_gender, y_gender, test_size=0.2, random_state=42
)



In [ ]:
# Build CNN model for Gender Detection
gender_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])



In [ ]:
#Compiling and training model
gender_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gender_model.fit(X_train_gender, y_train_gender, epochs=10, batch_size=32,
                 validation_data=(X_test_gender, y_test_gender))

# Evaluating the model
test_loss_gender, test_accuracy_gender = gender_model.evaluate(X_test_gender, y_test_gender)
print(f"Gender Classification Test Accuracy: {test_accuracy_gender * 100:.2f}%")

gender_model.save("gender_classifier.h5") #Save the model as .h5 file






In [ ]:
bearded_folder = "beard\Train1\Beard"  # Paths for beard dataset
beardless_folder = "beard\Train1\No Beard"

# Load beard images (1 for bearded, 0 for beardless)
bearded_images, bearded_labels = load_images_from_folder(bearded_folder, 1)
beardless_images, beardless_labels = load_images_from_folder(beardless_folder, 0)

# Combining beard data
X_beard = np.array(bearded_images + beardless_images)
y_beard = np.array(bearded_labels + beardless_labels)

# Splitting beard data
X_train_beard, X_test_beard, y_train_beard, y_test_beard = train_test_split(
    X_beard, y_beard, test_size=0.2, random_state=42
)

In [ ]:
# Build CNN model for Beard detection
beard_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
    ]
)
# Train beard model
beard_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
beard_model.fit(X_train_beard, y_train_beard, epochs=20, batch_size=32,
                validation_data=(X_test_beard, y_test_beard))

# Evaluatting beard model
test_loss_beard, test_accuracy_beard = beard_model.evaluate(X_test_beard, y_test_beard)
print(f"Beard Classification Test Accuracy: {test_accuracy_beard * 100:.2f}%")

beard_model.save("beard_classifier.h5")  # Save this model also as a .h5 file


In [ ]:
# Loading both models at the same time
gender_model = tf.keras.models.load_model("gender_classifier.h5")
beard_model = tf.keras.models.load_model("beard_classifier.h5")

sample_image_path = "gender.v1i.folder\train\Male\160103_jpg.rf.232281783be733885cec9224da42ec72.jpg"  #Sample image
img_resized = cv2.resize(img, (64, 64))
img_normalized = img_resized / 255.0
img_input = np.expand_dims(img_normalized, axis=0)

gender_pred = gender_model.predict(img_input)   #Predict Gender
gender = "Female" if gender_pred[0][0] > 0.5 else "Male"
print(f"Predicted gender: {gender}")


if gender == "Male":   #If its a male then predict Beard or no Beard
    beard_pred = beard_model.predict(img_input)
    beard_status = "Bearded" if beard_pred[0][0] > 0.5 else "Beardless"
    print(f"Predicted beard status: {beard_status}")
else:
    print("Beard classification not applicable for female.")